## Full Text Search / Information Retrieval 

## 1) : Why build an Inverted Index for text-retrieval?

## 2) : Objectives:
* Collect a corpus
* Perform a brute-force query and measure the performance
* Create a forward and inverted index from the above 
* Search on one or a combination of terms and measure performance

In [1]:
from tqdm import tqdm
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from collections import defaultdict
import time
import re

In [2]:
corpus = fetch_20newsgroups(remove=['header','footer'])['data']

In [3]:
corpus[0]

"From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n"

---

### Brute Force

#### Lets start with a example for one term query

In [7]:
re.findall('(?u)\\b\\w\\w+\\b',corpus[0])

['From',
 'lerxst',
 'wam',
 'umd',
 'edu',
 'where',
 'my',
 'thing',
 'Subject',
 'WHAT',
 'car',
 'is',
 'this',
 'Nntp',
 'Posting',
 'Host',
 'rac3',
 'wam',
 'umd',
 'edu',
 'Organization',
 'University',
 'of',
 'Maryland',
 'College',
 'Park',
 'Lines',
 '15',
 'was',
 'wondering',
 'if',
 'anyone',
 'out',
 'there',
 'could',
 'enlighten',
 'me',
 'on',
 'this',
 'car',
 'saw',
 'the',
 'other',
 'day',
 'It',
 'was',
 'door',
 'sports',
 'car',
 'looked',
 'to',
 'be',
 'from',
 'the',
 'late',
 '60s',
 'early',
 '70s',
 'It',
 'was',
 'called',
 'Bricklin',
 'The',
 'doors',
 'were',
 'really',
 'small',
 'In',
 'addition',
 'the',
 'front',
 'bumper',
 'was',
 'separate',
 'from',
 'the',
 'rest',
 'of',
 'the',
 'body',
 'This',
 'is',
 'all',
 'know',
 'If',
 'anyone',
 'can',
 'tellme',
 'model',
 'name',
 'engine',
 'specs',
 'years',
 'of',
 'production',
 'where',
 'this',
 'car',
 'is',
 'made',
 'history',
 'or',
 'whatever',
 'info',
 'you',
 'have',
 'on',
 'this'

In [10]:
def brute_force_query(query, corpus): 
    """Returns indices of matching documents"""
    doc_ids = []
    for i, doc in enumerate(corpus):
        for word in re.findall('(?u)\\b\\w\\w+\\b',doc.lower()):
            if word == query:
                doc_ids.append(i)
    return doc_ids

In [11]:
brute_force_query('bricklin', corpus)

[0, 958, 2835, 4988]

#### Ok lets measure the performance of this approach

In [12]:
%timeit brute_force_query('bricklin', corpus)

1.19 s ± 7.64 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


---

### Create a forward and an inverted Index 

In [16]:
def create_forward_index(corpus):
    forward_index = defaultdict(list)
    for i, doc in enumerate(corpus):
        forward_index[i] = re.findall('(?u)\\b\\w\\w+\\b',doc.lower()) #recognise this regex anyone?
    return forward_index

In [17]:
def create_inverted_index(corpus):
    index = defaultdict(list) 
    for i, doc in enumerate(corpus):
        for word in re.findall('(?u)\\b\\w\\w+\\b',doc.lower()):
            # index.setdefault(word, [])
            index[word].append(i)
    return index

#### Ok lets measure the performance of this approach

In [18]:
result = create_inverted_index(corpus)

In [19]:
%timeit create_inverted_index(corpus)

1.85 s ± 21.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Is this what we expected?

In [20]:
%timeit result['bricklin']

48.3 ns ± 0.455 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


---

### Rank the documents which contain the most references

In [13]:
def find_terms(query, index):
    query = query.lower().split()
    results = defaultdict(list) 
    for term in query:
        results[term] = index[term]
    return results

In [ ]:
find_terms(query, index)

---

### Other things to consider:

* Results ranking isn't as simple as finding all the results!!
    * e.g - which documents should come back first requires some page ranking - cf Markov and his buddies!!
    * e.g - intra-doc ranking - titles/links could be more important than words in text
    * e.g - intra-doc ranking - long documents will have more occurances of a term
* standard NLP: tokenization, stop words, language, remove crap, parsing, encoding
* typos: Levenshtein distance "hippo" - "hypo" -> Levenshtein dist == 2
* trigram search (heuristic algorithm)

---

### Technologies that utilize inverted-index from scratch:

* Postgres
* Mongodb
* Elasticsearch

---